In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
# https://discuss.pytorch.org/t/how-to-implement-keras-layers-core-lambda-in-pytorch/5903
class BestMeanPooling(nn.Module):
    
  def __init__(self, n_topN):
    super(BestMeanPooling, self).__init__()
    self.topN = n_topN
        
  def forward(self, aX_tr):
    """
    
    """
    aX_tr_sorted, _ = torch.sort(aX_tr, axis=1) # also return index
    return torch.mean(aX_tr_sorted[:, -self.topN:,:], axis=1)
    # print("B = ", aX_tr_sorted.size(), aX_tr_sorted)
    # c = torch.mean(aX_tr_sorted[:, -self.topN:,:], axis=1)
    # print("C = ", c.size(), c)
    return c

# Test
aA = torch.randn(1, 200).reshape(4, 10, 5)
mean_pool = BestMeanPooling(2) 
aA = mean_pool(aA)
# print(aA)

In [3]:
class SyntheticData(Dataset):
    def __init__(self, upper, x, y, z):
      self.aX_tr_sy = np.random.random_sample(upper).reshape(x, y, z)
      self.ay_tr_sy = np.random.randint(0, 2, x)

    def __len__(self):
        return len(self.aX_tr_sy)

    def __getitem__(self, idx):
        return self.aX_tr_sy[idx], self.ay_tr_sy[idx] #.squeeze(1)

In [4]:
class MassCNN(nn.Module):

  def __init__(self, n_in, n_out, num_event=10, topN=5, prob=0.00):
    super(MassCNN,self).__init__()
    self.n_filter = n_out

    #first layers
    self.conv_1        = nn.Conv1d(n_in, n_out, kernel_size=1)
    
    # output of the layer is # of filters
    self.bmpool_1      = BestMeanPooling(topN)
    self.dropout_L1    = nn.Dropout(p=prob)
    
    # Fully connected layer
    self.fcn_1 = nn.Linear(3, 2)
    # self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, aX_tr):
    aX_tr = self.conv_1(aX_tr)
    aX_tr = torch.relu(aX_tr)
    
    # This will collapse one dimension
    aX_tr = self.bmpool_1(aX_tr)
    
    if self.n_filter > 4: 
      aX_tr = self.dropout_L1(aX_tr)
    
    aX_tr = aX_tr.view(aX_tr.size(0), -1)
    aX_tr = self.fcn_1(aX_tr)
    # aX_tr = self.softmax(aX_tr)
    return aX_tr

In [5]:
def train(n_epochs=10, lr=0.1, filter=12, maxpool_percent=100.00, n_iter = 0):
    """
      Aim:

      params:

      returns:
    """
    # Random data
    d_trains = SyntheticData(750, 5, 50, 3)
    d_valids = SyntheticData(750, 5, 50, 3)

    o_mass_model = MassCNN(50, filter, 3, 5)
    
    # Trying to remove this type casting from here.
    # massModel = massModel #.float()
    
    # optimization
    o_optimizer = torch.optim.SGD(o_mass_model.parameters(), lr=0.0001)
    o_cost      = torch.nn.CrossEntropyLoss()
  
    t_cost_per_epoch = []
    
    # model.state_dict()['cnn1.weight']
    # model.state_dict()['cnn2.weight']
    af_Tr = DataLoader(d_trains, batch_size=100, shuffle=True)
    af_Va = DataLoader(d_valids, batch_size=10, shuffle=True)
    t_solutions = []
    for epoch in range(n_epochs):
      m_models = dict()
      fgr_cost = 0
      for af_X, af_y in af_Tr:  # iterative over batch of data
        ## Back propagation the loss
        o_optimizer.zero_grad()
        af_y_pr  = o_mass_model(af_X.float())
        o_loss   = o_cost(af_y_pr, af_y)
        o_loss.backward()
        o_optimizer.step() # back propagate
        fgr_cost += o_loss.data
        
      t_cost_per_epoch.append(fgr_cost)
      
      #print(o_mass_model.state_dict().keys())
      #print("Epochs in fun ", epoch, id(o_mass_model))
      o_mass_model.eval()
      f_correct = 0
      for af_x_va, af_y_va in af_Va:
        z  = o_mass_model(af_x_va.float()).data
        _, yhat = torch.max(z.data, 1)
        f_correct += (yhat == af_y_va).sum().item()
      f_accuracy = "%0.3f" %(f_correct / 1000) # fixed this with sample size
    
      m_models["epoch"] = epoch 
      m_models["iteration"] = n_iter
      m_models["val_acc"] = f_accuracy
      m_models["model_state"] =  o_mass_model.state_dict()
      m_models["optimizer_state"] =  o_optimizer.state_dict()
      m_models["loss"] =  o_cost.state_dict()
      m_models["model"] = o_mass_model
      t_solutions.append(m_models.copy())
      del m_models
    return t_solutions

In [6]:
def get_best_models(t_models):
  """
  Select the best solutions from the pool
  
  params: t_models - list of models. Each model is dictionary that contains parameter
                     and hyperparameters.
  """
  n_best_sol      = 5
  t_best_val_accs = []
  t_best_models   = []
    
  for o_model in t_models: 
    b_update  = True
    f_val_acc = float(o_model["val_acc"])
    if len(t_best_val_accs) >= n_best_sol:
      print(t_best_val_accs)
      f_lowest_acc = np.min(t_best_val_accs)
      if f_val_acc > f_lowest_acc:
        i_cur_idx = t_best_val_accs.index(f_lowest_acc)
        t_best_val_accs.pop(i_cur_idx)
        t_best_models.pop(i_cur_idx)
      else:
        b_update = False
    if b_update: 
      t_best_val_accs.append(f_val_acc)
      t_best_models.append(o_model)
  return t_best_models

In [7]:
# load the multiple models for testing

def test_model(d_tests, t_best_sols):
  """
  Test performance of model on independent data. Since multiple models were generated, 
  these models were tested aganist an independent data and their average performance 
  will be returned as final result.
  
  params:
    d_tests: data for independent test
    t_best_sols: list of best solutions
  """
  t_test_accs = []
  for o_best_sol in t_best_sols:
    o_best_model = o_best_sol["model"]
    o_best_model.eval()
    f_correct = 0
    n_test_count = len(d_tests)
    for d_test_X, d_target_Y in d_tests:
      z  = o_best_model(d_test_X.float()).data # remove float
      _, yhat = torch.max(z.data, 1)
      f_correct += (yhat == d_target_Y).sum().item()
    f_accuracy = "%0.3f" %(f_correct / n_test_count) # fixed this with sample size
    t_test_accs.append(float(f_accuracy))
  print("Coding: ", t_test_accs)
  return np.mean(t_test_accs)

# model.load_state_dict(checkpoint['model_state_dict'])

In [8]:
# get 20% of best solutions after iterating 
# between 15 to 30 times (these # are randomly selected)

n_cell = 10
f_maxpools    = [0.01, 1.0, 5.0, 20.0, 100.0]
n_maxpool_len = len(f_maxpools)

t_models      = []
for n_trial in range(10):
  f_lr       = 10 ** np.random.uniform(-3, -2)
  i_filter   = np.random.choice(range(3,10))
  f_max_pool = f_maxpools[n_trial%n_maxpool_len]
  f_maxpool  = max(1, int(f_max_pool/100. * n_cell))
  t_models += train(10, f_lr, i_filter, f_maxpool)

t_best_models = get_best_models(t_models)

# generate test data
d_tests  = SyntheticData(750, 5, 50, 3)
af_test = DataLoader(d_tests)
predicted_Y = test_model(af_test, t_best_models)
print("Coding: ", predicted_Y)

[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.003]
[0.003, 0.003, 0.003, 0.003, 0.004]
[0.003, 0.003, 0.003, 0.004, 0.004]
[0.003, 0.003, 0.004, 0.004, 0.004]
[0.003, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004, 0.004]
[0.004, 0.004, 0.004, 0.004,

### Keras

In [9]:
from keras.layers import Input, Dense, Lambda, Activation, Dropout
from keras.layers.convolutional import Convolution1D
from keras.models import Model
from keras.optimizers import Adam

# the input layer
ncell = 20
nmark = 5
data_input = Input(shape=(ncell, nmark))

nfilter = 10
# the filters
print(data_input.shape)
conv = Convolution1D(nfilter, 1, activation='linear', name='conv1')(data_input)
print(conv.shape)


import keras.backend as K
K.print_tensor(conv)

Using TensorFlow backend.


(None, 20, 5)
(None, 20, 10)


<tf.Tensor 'Identity:0' shape=(None, 20, 10) dtype=float32>

In [10]:
a = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=7, stride=1, padding=3)

In [11]:
"""
in_channel = 16
out_channel = 33
filter/kernal = 3
batch = 20
"""

m = nn.Conv1d(3, 6, 2, stride=1)
input = torch.randn(5, 3, 4)
print(input.shape)
output = m(input)
print(output.shape)

torch.Size([5, 3, 4])
torch.Size([5, 6, 3])


In [12]:
output.shape

torch.Size([5, 6, 3])

In [13]:
input

tensor([[[ 0.6264, -1.0689,  0.8947,  0.6613],
         [-0.0609, -0.1427,  0.6524, -0.6114],
         [ 0.8502, -1.1811,  0.2023, -0.1698]],

        [[ 1.9426, -0.3940,  0.1056, -0.0188],
         [ 0.3037, -0.2814,  0.5930,  0.8171],
         [-2.2596, -1.3615,  0.1173, -0.7407]],

        [[ 1.2895,  0.9866,  0.4093,  0.0553],
         [-1.3481,  2.8548,  0.0110, -1.6774],
         [ 0.3700, -0.9349,  1.0409, -0.3617]],

        [[ 0.5112, -1.8242, -1.7307,  0.7217],
         [-0.0374,  0.0032,  0.2039, -0.5883],
         [ 0.5928, -0.6295, -1.2091, -1.3084]],

        [[-1.9813, -1.3394, -0.0769, -0.7997],
         [ 0.2150,  1.5719, -2.7364,  0.4307],
         [ 0.1438,  0.9363, -1.3732, -0.2300]]])

In [14]:
output

tensor([[[ 0.9761,  0.0494,  0.2616],
         [-0.1339,  0.6016,  0.2954],
         [-0.6942, -0.1283,  0.2529],
         [ 0.1993, -0.4620,  0.0325],
         [ 0.2876,  0.4290, -0.1779],
         [-0.5350,  0.9385, -0.0227]],

        [[-0.2537, -0.0465,  0.3789],
         [-0.3155,  0.3745, -0.0300],
         [ 0.5402, -0.1488, -0.4576],
         [ 0.0028, -0.2816,  0.0329],
         [ 1.2074,  0.7800,  0.0326],
         [ 0.3150,  0.5926,  0.3196]],

        [[-1.0598, -0.3572,  1.1398],
         [-1.1925,  0.6258,  0.5860],
         [-1.1714,  0.9764, -0.0204],
         [-1.0419,  0.8299,  0.0927],
         [-0.2879, -0.1128, -0.0728],
         [ 0.7170,  0.3465, -0.6416]],

        [[ 0.9160,  1.6411,  1.2504],
         [-0.0197,  0.5581,  0.8467],
         [-0.7527, -0.8986, -0.0225],
         [ 0.4419,  0.2732, -0.3323],
         [ 0.6353,  1.1966,  0.6019],
         [-0.7396,  0.1547,  0.9562]],

        [[ 0.6942,  2.6339, -0.0306],
         [ 0.6850,  1.3378, -0.0089],
    

In [15]:
from heapq import heappush, heappop
heap = []
data = [(1, 'J'), (4, 'N'), (3, 'H'), (2, 'O')]
for item in data:
  heappush(heap, item)
  print("item ", item, "heap ", heap)

while heap:
  print(heappop(heap)[1])

item  (1, 'J') heap  [(1, 'J')]
item  (4, 'N') heap  [(1, 'J'), (4, 'N')]
item  (3, 'H') heap  [(1, 'J'), (4, 'N'), (3, 'H')]
item  (2, 'O') heap  [(1, 'J'), (2, 'O'), (3, 'H'), (4, 'N')]
J
O
H
N


In [16]:
import numpy as np
a = list(np.random.uniform(10, 15, 10))
np.min(a)

11.087687378546482

In [17]:
a

[13.019949284190336,
 11.087687378546482,
 14.139058826352988,
 13.207704703872293,
 11.350121239849562,
 14.330643797103484,
 11.649265839287658,
 12.824486314689969,
 11.334979842012935,
 11.6434292321006]